In [1]:
from agent_framework import MCPStdioTool, MCPStreamableHTTPTool, MCPStdioTool


In [2]:
tool = MCPStreamableHTTPTool(
            name="Microsoft Learn MCP",
            url="https://learn.microsoft.com/api/mcp",
            # we don't require approval for microsoft_docs_search tool calls
            # but we do for any other tool
            # approval_mode={"never_require_approval": ["microsoft_docs_search"]},
        )

In [3]:
tool2 = MCPStdioTool(  
    name="filesystem",  
    command="npx",  
    args=["-y", "@modelcontextprotocol/server-filesystem", r"C:\Users\blain\Documents"],  
    description="File system operations",  
)

In [4]:
tool3 = MCPStreamableHTTPTool(
            name="localhost MCP",
            url="http://localhost:1337/mcp",
        )


In [ ]:
import logging
import httpx
from httpx._content import ByteStream

# Restore any previous request patch from earlier runs
if "original_request" in globals():
    httpx.AsyncClient.request = original_request
    del globals()["original_request"]

# Reset root handlers and silence everything except our custom httpx logs
root_logger = logging.getLogger()
for handler in list(root_logger.handlers):
    root_logger.removeHandler(handler)
logging.basicConfig(level=logging.ERROR)

# Limit output to debug entries from httpx.logged_send_single_request
class _HttpxFunctionFilter(logging.Filter):
    def filter(self, record: logging.LogRecord) -> bool:
        return record.funcName == "logged_send_single_request"

httpx_logger = logging.getLogger("httpx")
httpx_logger.handlers.clear()
httpx_logger.setLevel(logging.DEBUG)
httpx_logger.propagate = False

httpx_handler = logging.StreamHandler()
httpx_handler.setLevel(logging.DEBUG)
httpx_handler.setFormatter(logging.Formatter('\n[%(name)s - %(module)s] %(message)s'))
httpx_handler.addFilter(_HttpxFunctionFilter())
httpx_logger.addHandler(httpx_handler)


# Also enable openai library logging if available
# logging.getLogger("openai").setLevel(logging.DEBUG)

if not hasattr(httpx.AsyncClient, "_original_send_single_request"):
    httpx.AsyncClient._original_send_single_request = httpx.AsyncClient._send_single_request

async def logged_send_single_request(self, request):
    logger = logging.getLogger("httpx")
    if logger.isEnabledFor(logging.DEBUG):
        stream = request.stream
        if isinstance(stream, ByteStream):
            body = stream._stream
            logger.debug("Request: %s %s", request.method, request.url)
            if body:
                try:
                    logger.debug("\tRequest body (utf-8): %s", body.decode("utf-8"))
                except UnicodeDecodeError:
                    logger.debug("\tRequest body (bytes): %s", body)
            else:
                logger.debug("\tRequest body: <empty>")
        else:
            logger.debug("\tRequest body not logged (stream type: %s)", type(stream).__name__)
    response = await httpx.AsyncClient._original_send_single_request(self, request)
    if logger.isEnabledFor(logging.DEBUG):
        # logger.debug("Response headers: %s", dict(response.headers))
        logger.debug(f"\tResponse {response.status_code}: {await response.aread()}")
    return response

if not getattr(httpx.AsyncClient, "_logging_patch_applied", False):
    httpx.AsyncClient._send_single_request = logged_send_single_request
    httpx.AsyncClient._logging_patch_applied = True

from agent_framework.openai import OpenAIChatClient

# tool = MCPStreamableHTTPTool(
#             name="localhost MCP",
#             url="http://localhost:1337/mcp",
#         )
llm = OpenAIChatClient(
        api_key="ollama", # Just a placeholder, Ollama doesn't require API key
        # base_url="http://localhost:11434/v1",
        base_url="http://ollama.home/v1",
        model_id="gpt-oss:20b",
    )
agent = llm.create_agent(
        name="test_agent",
        instructions="You are a helpful agent. You use Model Context Protocol (MCP) tools to answer user questions. "\
            "You can only respond using the tools available to you. Do not make up tool functionality. The tools will be" \
                "Provided to you in the prompt.",
        tools=[tool3],
    )

query = "hello."
print(f"User: {query}")
print("\n=== HTTP Request/Response Details Below ===\n")
result = await agent.run(query)
print(f"\n=== End of HTTP Details ===\n")
print(f"Agent: {result}\n")


User: hello.

=== HTTP Request/Response Details Below ===




[httpx - 984351430] Request: POST http://localhost:1337/mcp

[httpx - 984351430] 	Request body (utf-8): {"method":"initialize","params":{"protocolVersion":"2025-06-18","capabilities":{"sampling":{}},"clientInfo":{"name":"mcp","version":"0.1.0"}},"jsonrpc":"2.0","id":0}

[httpx - 984351430] 	Response 200: b'event: message\r\ndata: {"jsonrpc":"2.0","id":0,"result":{"protocolVersion":"2025-06-18","capabilities":{"experimental":{},"prompts":{"listChanged":true},"resources":{"subscribe":false,"listChanged":true},"tools":{"listChanged":true}},"serverInfo":{"name":"uber-mcp","version":"1.14.1"}}}\r\n\r\n'

[httpx - 984351430] Request: POST http://localhost:1337/mcp

[httpx - 984351430] 	Request body (utf-8): {"method":"notifications/initialized","jsonrpc":"2.0"}

[httpx - 984351430] Request: GET http://localhost:1337/mcp

[httpx - 984351430] 	Request body: <empty>

[httpx - 984351430] 	Response 202: b''

[httpx - 984351430] Request: POST http://localhost:1337/mcp

[httpx - 984351430] 	Reques

CancelledError: 


[httpx - 984351430] 	Response 200: b'event: message\r\ndata: {"jsonrpc":"2.0","id":3,"result":{"content":[{"type":"text","text":"..."}],"structuredContent":{"result":"..."},"isError":false}}\r\n\r\n'
ERROR:agent_framework._mcp:Error from MCP server: Received response with an unknown request ID: SessionMessage(message=JSONRPCMessage(root=JSONRPCResponse(jsonrpc='2.0', id=3, result={'content': [{'type': 'text', 'text': '...'}], 'structuredContent': {'result': '...'}, 'isError': False})), metadata=None)
RuntimeError: Received response with an unknown request ID: SessionMessage(message=JSONRPCMessage(root=JSONRPCResponse(jsonrpc='2.0', id=3, result={'content': [{'type': 'text', 'text': '...'}], 'structuredContent': {'result': '...'}, 'isError': False})), metadata=None)


In [ ]:
import asyncio
import os
from random import randint
from typing import Annotated

from agent_framework.openai import OpenAIChatClient

"""
Ollama with OpenAI Chat Client Example

This sample demonstrates using Ollama models through OpenAI Chat Client by
configuring the base URL to point to your local Ollama server for local AI inference.
Ollama allows you to run large language models locally on your machine.

Environment Variables:
- OLLAMA_ENDPOINT: The base URL for your Ollama server (e.g., "http://localhost:11434/v1/v1/")
- OLLAMA_MODEL: The model name to use (e.g., "mistral", "llama3.2", "phi3")
"""


def get_weather(
    location: Annotated[str, "The location to get the weather for."],
) -> str:
    """Get the weather for a given location."""
    conditions = ["sunny", "cloudy", "rainy", "stormy"]
    return f"The weather in {location} is {conditions[randint(0, 3)]} with a high of {randint(10, 30)}°C."


async def non_streaming_example() -> None:
    """Example of non-streaming response (get the complete result at once)."""
    print("=== Non-streaming Response Example ===")

    agent = llm.create_agent(
        name="WeatherAgent",
        instructions="You are a helpful weather agent.",
        tools=get_weather,
    )

    query = "What's the weather like in Seattle?"
    print(f"User: {query}")
    result = await agent.run(query)
    print(f"Agent: {result}\n")


async def streaming_example() -> None:
    """Example of streaming response (get results as they are generated)."""
    print("=== Streaming Response Example ===")

    agent = llm.create_agent(
        name="WeatherAgent",
        instructions="You are a helpful weather agent.",
        tools=get_weather,
    )

    query = "What's the weather like in Portland?"
    print(f"User: {query}")
    print("Agent: ", end="", flush=True)
    async for chunk in agent.run_stream(query):
        if chunk.text:
            print(chunk.text, end="", flush=True)
    print("\n")

In [ ]:
await streaming_example()